In [1]:
import overpy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

def geocode_address(address):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)


def get_nearby_amenities(address):
    try:
        coordinates = geocode_address(address)

        overpass_query = f"""
            node(around:500, {coordinates[0]}, {coordinates[1]})
            ["amenity"];
            out;
        """

        api = overpy.Overpass()
        result = api.query(overpass_query)

        amenities = [node.tags.get('amenity', 'N/A') for node in result.nodes]
        return ", ".join(amenities)
    
    except Exception as e:
        return str(e)

def process_excel(input_file, output_file):
    df = pd.read_excel(input_file)

    df['Amenities'] = df['Address'].apply(get_nearby_amenities)

    df.to_csv(output_file, index=False)

# Example usage
input_excel_file = "input_file.xlsx"
output_csv_file = "output_amenities.csv"
process_excel(input_excel_file, output_csv_file)


/Users/ellasartory/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Create data frame
csv = pd.read_csv("output_amenities.csv", encoding='utf-8')
amt = pd.DataFrame(csv)

# Extracting all categories from amenities 
amenities_mentioned = amt["Amenities"].str.split().explode().unique().tolist()

# Print list of unique amenities
print(amenities_mentioned)

['post_box,', 'recycling,', 'fast_food,', 'restaurant,', 'police,', 'bar,', 'fire_station,', 'clinic,', 'cafe,', 'bank,', 'bicycle_parking,', 'parking,', 'toilets,', 'lockers,', 'bench,', 'drinking_water,', 'bicycle_rental,', 'parking_entrance,', 'vending_machine,', 'piano,', 'charging_station,', 'doctors,', 'clock,', 'waste_basket,', 'boat_rental,', 'public_bookcase,', 'vending_machine', 'fuel,', 'motorcycle_parking,', 'bench', 'pharmacy,', 'pub,', 'waste_disposal,', 'ice_cream,', 'studio,', 'community_centre,', 'place_of_worship,', 'bbq,', 'taxi,', 'fountain,', 'social_facility,', 'shelter,', 'arts_centre,', 'theatre,', 'dentist,', 'kindergarten,', 'school,', 'food_court,', 'atm,', 'parking_entrance', 'townhall,', 'healthcare,', 'social_centre,', 'dental_hygienist,', 'cinema,', 'fountain', 'bureau_de_change,', 'bus_station,', 'nightclub,', 'casino,', 'coworking_space,', 'accupuncture,', 'brothel,', 'bicycle_repair_station,', 'cannabis,', 'animal_shelter,', 'car_rental,', 'ferry_termi

In [3]:
import pandas as pd

excel = "google_maps_reviews_english_googletrans.xlsx"

# Read Excel file into DataFrame
rating_amenities = pd.read_excel(excel)

# Drop specific columns from the original DataFrame
rating_amenities = rating_amenities.drop(["name", "review_text", "location_link", "reviews_total", "review_datetime_utc", "reviews_per_score_1", "reviews_per_score_2", "reviews_per_score_3", "reviews_per_score_4", "reviews_per_score_5", "review_text_english_googletrans"], axis=1)

# Display the modified DataFrame
print(rating_amenities)


                         place_id  rating  review_rating
0     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              2
1     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              5
2     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              1
3     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              1
4     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              5
...                           ...     ...            ...
1210  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1211  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1212  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1213  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1214  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5

[1215 rows x 3 columns]


In [4]:
amenity_counts_dict = {amenity: [] for amenity in amenities_mentioned}

# Loop through each value in the specified column
for amenities_str in amt["Amenities"]:
    # Loop through each target amenity
    for amenity in amenities_mentioned:
        # Count the occurrences of the current target amenity in the current cell
        count = amenities_str.count(amenity)
        
        # Append the count to the corresponding list in the dictionary
        amenity_counts_dict[amenity].append(count)

# Add the counts as new columns in the DataFrame
for amenity, counts in amenity_counts_dict.items():
    amt[f'{amenity}_counts'] = counts

# Add a new column based on an existing column
# For example, let's create a column 'total_counts' which sums up the counts for all amenities
amt['total_counts'] = amt[[f'{amenity}_counts' for amenity in amenities_mentioned]].sum(axis=1)

# Display the modified DataFrame
display(amt)


/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_46759/2986390775.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_46759/2986390775.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_46759/2986390775.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

,Address,Amenities,"post_box,_counts","recycling,_counts","fast_food,_counts","restaurant,_counts","police,_counts","bar,_counts","fire_station,_counts","clinic,_counts",...,"parcel_locker,_counts","events_venue,_counts",charging_station_counts,"ticket_validator,_counts","photo_booth,_counts","hospital,_counts",fuel_counts,"playground,_counts",balance_beam_counts,total_counts
0,"Julianaplein 1, Amsterdam","post_box, recycling, fast_food, restaurant, re...",3,13,3,7,1,1,1,1,...,0,0,2,0,0,0,0,0,0,175
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam","parking, post_box, parking, fuel, charging_sta...",1,0,0,0,0,0,0,0,...,0,0,5,0,0,0,1,0,0,29
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam","fuel, post_box, fast_food, pharmacy, pub, pub,...",2,16,5,6,0,0,0,0,...,0,0,0,0,0,0,1,0,0,193
3,"Spoorslag 29, 1082 MM Amsterdam","cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,0,2,0,0,...,0,0,1,0,0,0,0,0,0,233
4,"Julianaplein Oost, 1097 DN Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,15,4,8,1,1,1,1,...,0,0,2,0,0,0,0,0,0,195
5,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,0,3,0,0,0,0,0,0,232
6,"Stationsplein 39, 1012 AB Amsterdam","restaurant, pub, pub, pub, restaurant, bureau_...",1,6,59,120,0,24,0,0,...,0,0,0,0,0,0,0,0,0,681
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam","recycling, recycling, recycling, post_box, res...",3,29,5,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,161
8,"Europaboulevard 4A, 1083 AD Amsterdam","theatre, conference_centre, parking, parking, ...",1,1,0,3,0,0,0,1,...,0,0,0,0,0,0,0,0,0,150
9,"Kattenburgerstraat 5, 1018 JA Amsterdam","pub, post_box, restaurant, restaurant, restaur...",3,5,2,18,0,0,1,0,...,0,0,5,0,0,0,0,0,0,248


In [5]:
#manually add place id column to be able to merge with rating_df

def map_amenities(value):
    # Example: If 'pool' is in the value, set the new column to 'water_feature'
    if 'Julianaplein 1, Amsterdam' in value:
        return 'ChIJ7392nH8JxkcRM5jumNJkgFw'
    # Example: If 'gym' is in the value, set the new column to 'exercise'
    elif 'Joan Muyskenweg 22, 1096 CJ Amsterdam' in value:
        return 'ChIJSaEgtTIKxkcRUgPJeDmIzJI'
    # Example: If 'wifi' is in the value, set the new column to 'internet'
    elif 'Nieuwe Leeuwarderweg, 1021 BZ Amsterdam' in value:
        return 'EixOaWV1d2UgTGVldXdhcmRlcndlZywgQW1zdGVyZGFtLCBOZXRoZXJsYW5kcyIuKiwKFAoSCa33Xuz2CMZHEQ6g0ognO2_oEhQKEglVd5qUtT_GRxGNr8C3TP0AZg'
    elif 'Spoorslag 29, 1082 MM Amsterdam' in value:
        return 'ChIJjwDOAQUKxkcRr9PeeaTkB7k'
    elif 'Julianaplein Oost, 1097 DN Amsterdam' in value:
        return 'EixKdWxpYW5hcGxlaW4sIDEwOTcgRE4gQW1zdGVyZGFtLCBOZXRoZXJsYW5kcyIuKiwKFAoSCSsmR59_CcZHEe0uZ5Hg4TcpEhQKEglf-SeZfwnGRxFlD_cllxIq-g'
    elif 'Termini 23, Amsterdam' in value:
        return 'ChIJn0fdmPwJxkcRYe7a-6JHfpI'
    elif 'Stationsplein 39, 1012 AB Amsterdam' in value:
        return ' ChIJycsDvbcJxkcRqx8XcA3ijg4'
    elif 'Cornelis Lelylaan 35, 1062 HD Amsterdam' in value:
        return 'ChIJ71V5XiPixUcRSKlH5OYy-iE'
    elif 'Europaboulevard 4A, 1083 AD Amsterdam' in value:
        return 'ChIJLWMViCIKxkcRi1ThzDUCzCo'
    elif 'Kattenburgerstraat 5, 1018 JA Amsterdam' in value:
        return 'ChIJhWKhkaYJxkcR1kC1yPyTJEA'
    elif 'Papaverweg 37, 1032 KK Amsterdam' in value:
        return 'ChIJY1m7LU8IxkcRMTbZnrKsPvo'
    elif 'Tt. Vasumweg 32, 1033 SC Amsterdam' in value:
        return 'ChIJG1OVEJ8JxkcRbAP_mgCGLaI'
    elif 'Benjamin Brittenstraat 13, 1082 ML Amsterdam' in value:
        return 'ChIJXbKy8gUKxkcRSPJc_oqBJwA'
    elif 'Haringbuisdijk 643, 1086 VA Amsterdam' in value:
        return 'ChIJMbQArgMPxkcRSlcWaJQd7kI'
    elif 'James Wattstraat 77-K, 1097 DL Amsterdam' in value:
        return 'EjBKYW1lcyBXYXR0c3RyYWF0LCAxMDk3IERMIEFtc3RlcmRhbSwgTmV0aGVybGFuZHMiLiosChQKEgk3UtoieQnGRxHac0petjaGtBIUChIJ4YXxVn8JxkcRco7Tb1reABM'
    elif 'Pedro de Medinalaan 1, 1086 XK Amsterdam' in value:
        return 'ChIJE7cGdMoOxkcReVn1gUqpE14'
    elif 'Govert Flinckstraat 309 H, 1074 CA Amsterdam' in value:
        return 'ChIJX_a1lJEJxkcRk30L9oOgzFI'
    elif 'Van Vollenhovenstraat 5g, 3016 BE Rotterdam' in value:
        return 'ChIJxQpq9mEzxEcRDVnXZbO6ALc'
    elif 'Kolk 172, 3011 MD Rotterdam' in value:
        return 'ChIJbY0fn1wzxEcRCGYhc0Mnzpot'
    elif 'Stationsplein 298, 3013 AK Rotterdam' in value:
        return 'ChIJp9BsSqQ0xEcRFOhB0IHpArc'
    elif 'Stationsplein 2, 2611 BV Delft' in value:
        return 'ChIJEXul8cC1xUcR3mv07a-M6WM'
    elif 'Houttuinen, 2613 SL Delft' in value:
        return 'EiZIb3V0dHVpbmVuLCAyNjExIEFaIERlbGZ0LCBOZXRoZXJsYW5kcyIuKiwKFAoSCb8BCznBtcVHESWY3NSOandwEhQKEgn9WF9Rw7XFRxE91EvFo1ywiQ'
    elif 'Kon. Julianaplein 10, 2595 AA Den Haag' in value:
        return 'ChIJnSGImBe3xUcRtFY868Y8hcc'
    elif 'Van Embdenstraat 658, 2628 CN Delft' in value:
        return 'ChIJhzaXYYy1xUcRMZ2jasrWGjs'
    elif 'Tandwielstraat, 3083 AV Rotterdam' in value:
        return 'Ei5UYW5kd2llbHN0cmFhdCwgMzA4MyBBViBSb3R0ZXJkYW0sIE5ldGhlcmxhbmRzIi4qLAoUChIJ28uhjZgzxEcRnswwRAZjjEYSFAoSCQNHdpGYM8RHEcO2rY1b3gAd'
    elif 'Wilhelminakade 101, 3072 AP Rotterdam' in value:
        return 'ChIJq2DGt24zxEcR0QYMEoeB-0U'
    elif 'Busplein 16, 1315 KR Almere' in value:
        return 'ChIJCyuG6B8XxkcRHbDG-BedlEw'
    elif 'Stationsplein 131, 3511 ED Utrecht' in value:
        return 'ChIJ80xHI11vxkcRLeEDCzlh-ZU'
    elif 'Universiteitsweg, 3584 CT Utrecht' in value:
        return 'ChIJLcD8VY5oxkcRhmc-CiVXwjQ'
    elif 'Griffioenlaan 1BIS, 3526 LA Utrecht' in value:
        return 'EitHcmlmZmlvZW5sYWFuLCAzNTI2IExBIFV0cmVjaHQsIE5ldGhlcmxhbmRzIi4qLAoUChIJ08C2e8BlxkcRNpdQJr6TJCASFAoSCfHzuXnAZcZHEY0xEJoY7L_h'
    elif '1506 CK Zaandam' in value:
        return 'ChIJzeSVDNL8xUcRLXzFyb6NK2Y'
    # Add more conditions as needed
    else:
        return 'other'

# Apply the mapping function to create a new column
amt["place_id"] = amt["Address"].apply(map_amenities)

display(amt)

/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_46759/912065528.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt["place_id"] = amt["Address"].apply(map_amenities)


,Address,Amenities,"post_box,_counts","recycling,_counts","fast_food,_counts","restaurant,_counts","police,_counts","bar,_counts","fire_station,_counts","clinic,_counts",...,"events_venue,_counts",charging_station_counts,"ticket_validator,_counts","photo_booth,_counts","hospital,_counts",fuel_counts,"playground,_counts",balance_beam_counts,total_counts,place_id
0,"Julianaplein 1, Amsterdam","post_box, recycling, fast_food, restaurant, re...",3,13,3,7,1,1,1,1,...,0,2,0,0,0,0,0,0,175,ChIJ7392nH8JxkcRM5jumNJkgFw
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam","parking, post_box, parking, fuel, charging_sta...",1,0,0,0,0,0,0,0,...,0,5,0,0,0,1,0,0,29,ChIJSaEgtTIKxkcRUgPJeDmIzJI
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam","fuel, post_box, fast_food, pharmacy, pub, pub,...",2,16,5,6,0,0,0,0,...,0,0,0,0,0,1,0,0,193,EixOaWV1d2UgTGVldXdhcmRlcndlZywgQW1zdGVyZGFtLC...
3,"Spoorslag 29, 1082 MM Amsterdam","cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,0,2,0,0,...,0,1,0,0,0,0,0,0,233,ChIJjwDOAQUKxkcRr9PeeaTkB7k
4,"Julianaplein Oost, 1097 DN Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,15,4,8,1,1,1,1,...,0,2,0,0,0,0,0,0,195,EixKdWxpYW5hcGxlaW4sIDEwOTcgRE4gQW1zdGVyZGFtLC...
5,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,3,0,0,0,0,0,0,232,ChIJn0fdmPwJxkcRYe7a-6JHfpI
6,"Stationsplein 39, 1012 AB Amsterdam","restaurant, pub, pub, pub, restaurant, bureau_...",1,6,59,120,0,24,0,0,...,0,0,0,0,0,0,0,0,681,ChIJycsDvbcJxkcRqx8XcA3ijg4
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam","recycling, recycling, recycling, post_box, res...",3,29,5,3,0,0,0,0,...,0,0,0,0,0,0,0,0,161,ChIJ71V5XiPixUcRSKlH5OYy-iE
8,"Europaboulevard 4A, 1083 AD Amsterdam","theatre, conference_centre, parking, parking, ...",1,1,0,3,0,0,0,1,...,0,0,0,0,0,0,0,0,150,ChIJLWMViCIKxkcRi1ThzDUCzCo
9,"Kattenburgerstraat 5, 1018 JA Amsterdam","pub, post_box, restaurant, restaurant, restaur...",3,5,2,18,0,0,1,0,...,0,5,0,0,0,0,0,0,248,ChIJhWKhkaYJxkcR1kC1yPyTJEA


In [6]:
#merge 
merged_df = pd.merge(amt, rating_amenities, on = "place_id", how = 'left')

display(merged_df)

,Address,Amenities,"post_box,_counts","recycling,_counts","fast_food,_counts","restaurant,_counts","police,_counts","bar,_counts","fire_station,_counts","clinic,_counts",...,"ticket_validator,_counts","photo_booth,_counts","hospital,_counts",fuel_counts,"playground,_counts",balance_beam_counts,total_counts,place_id,rating,review_rating
0,"Julianaplein 1, Amsterdam","post_box, recycling, fast_food, restaurant, re...",3,13,3,7,1,1,1,1,...,0,0,0,0,0,0,175,ChIJ7392nH8JxkcRM5jumNJkgFw,NaN,NaN
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam","parking, post_box, parking, fuel, charging_sta...",1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,29,ChIJSaEgtTIKxkcRUgPJeDmIzJI,NaN,NaN
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam","fuel, post_box, fast_food, pharmacy, pub, pub,...",2,16,5,6,0,0,0,0,...,0,0,0,1,0,0,193,EixOaWV1d2UgTGVldXdhcmRlcndlZywgQW1zdGVyZGFtLC...,NaN,NaN
3,"Spoorslag 29, 1082 MM Amsterdam","cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,0,2,0,0,...,0,0,0,0,0,0,233,ChIJjwDOAQUKxkcRr9PeeaTkB7k,NaN,NaN
4,"Julianaplein Oost, 1097 DN Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,15,4,8,1,1,1,1,...,0,0,0,0,0,0,195,EixKdWxpYW5hcGxlaW4sIDEwOTcgRE4gQW1zdGVyZGFtLC...,NaN,NaN
5,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,0,0,0,0,0,232,ChIJn0fdmPwJxkcRYe7a-6JHfpI,3.8,3.0
6,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,0,0,0,0,0,232,ChIJn0fdmPwJxkcRYe7a-6JHfpI,3.8,1.0
7,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,0,0,0,0,0,232,ChIJn0fdmPwJxkcRYe7a-6JHfpI,3.8,5.0
8,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,0,0,0,0,0,232,ChIJn0fdmPwJxkcRYe7a-6JHfpI,3.8,5.0
9,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,0,1,...,0,0,0,0,0,0,232,ChIJn0fdmPwJxkcRYe7a-6JHfpI,3.8,5.0
